In [1]:
import numpy as np
import random
import copy

class TestGame:
    """
    A simple example game to use for q learning
    The possible amount of states for this are (shape[0]*shape[1])**2
    e.g.: With shape of (4, 4) will have "256" states, minus 16 for states where player and end are the same location
    In this case the action space is up, down, left and right so only 4
    
    Args:
        shape (tuple): the size of the grid to play on
    """
    def __init__(
        self,
        shape = (4,4)
    ):
        """
        Starting args
        Board for use in drawing
        start_pos to log the starting position for when drawing
        end_pos to log where the end position is for both drawing and finishing
        initialises both randomly until not the same
        actions dictionary used for when selection an action using argmax in the q learning step
        """
        self.board = [[' ' for j in range(shape[1])] for i in range(shape[0])]
        self.start_pos = (random.randint(0, shape[0] - 1), random.randint(0, shape[0] - 1))
        self.end_pos = self.start_pos
        while self.end_pos == self.start_pos:
            self.end_pos = (random.randint(0, shape[0] - 1), random.randint(0, shape[0] - 1))
        
        self.player_pos = list(self.start_pos)
        self.done = False
        self.actions = {
            0: self.up,
            1: self.down,
            2: self.left,
            3: self.right
        }
        
    def pos_check(self):
        if tuple(self.player_pos) == self.end_pos:
            self.done = True
            
    def up(self):
        if self.player_pos[0] != 0:
            self.player_pos[0] -= 1
            self.pos_check()
            
    def down(self):        
        if self.player_pos[0] != len(self.board) - 1:
            self.player_pos[0] += 1
            self.pos_check()
            
    def left(self):
        if self.player_pos[1] != 0:
            self.player_pos[1] -= 1
            self.pos_check()
            
    def right(self):        
        if self.player_pos[1] != len(self.board[0]) - 1:
            self.player_pos[1] += 1
            self.pos_check()
            
    def draw(self):
        board = copy.deepcopy(self.board)
        board[self.start_pos[0]][self.start_pos[1]] = 'S'
        board[self.end_pos[0]][self.end_pos[1]] = 'F'
        board[self.player_pos[0]][self.player_pos[1]] = 'P'
        for row in board:
            print(row)

In [2]:
import ipywidgets as w
from IPython.display import clear_output

"""
A simple widget based example to let the game be played

Provides directional buttons and a basic print of the board

Finishes when player lands on finished tile
"""

playable_game = TestGame(shape=(4,4))

gui = w.GridspecLayout(n_rows=2, n_columns=3)

def up_btn(Btn):
    playable_game.up()
    clear_output()
    if playable_game.done:
        playable_game.draw()
        print('\n\nYou win! ')
    else:
        display(gui)
        playable_game.draw()
def left_btn(Btn):
    playable_game.left()
    clear_output()
    if playable_game.done:
        playable_game.draw()
        print('\n\nYou win! ')
    else:
        display(gui)
        playable_game.draw()
def right_btn(Btn):
    playable_game.right()
    clear_output()
    if playable_game.done:
        playable_game.draw()
        print('\n\nYou win! ')
    else:
        display(gui)
        playable_game.draw()
def down_btn(Btn):
    playable_game.down()
    clear_output()
    if playable_game.done:
        playable_game.draw()
        print('\n\nYou win! ')
    else:
        display(gui)
        playable_game.draw()
    
up_button = w.Button(description='Up')
left_button = w.Button(description='Left')
right_button = w.Button(description='Right')
down_button = w.Button(description='Down')

up_button.on_click(up_btn)
left_button.on_click(left_btn)
right_button.on_click(right_btn)
down_button.on_click(down_btn)

gui[1,0] = left_button
gui[1,1] = down_button
gui[1,2] = right_button
gui[0,1] = up_button

display(gui)
playable_game.draw()

GridspecLayout(children=(Button(description='Left', layout=Layout(grid_area='widget001'), style=ButtonStyle())…

[' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', 'P']


In [3]:
SHAPE = (4,4)
LEARNING_RATE = 0.25
DISCOUNT_RATE = 0.75

MAX_EPISODE_LENGTH = 10
EPOCHS=1000

class GameLearner:
    """
    The class to handle q learning for the above game
    
    Args:
        game_shape (tuple): the shape to start each game instance for training, uses this for generating a Q table as well
        learing_rate (float): The value for alpha in the Q learning formula
        discount_rate (float): The value for gamma in the Q learning formula
        
    Example:
    
    q_learning_class = GameLearner()
    q_learning_class.fit_q_table(epochs=1000, steps_per_episode=MAX_EPISODE_LENGTH)
    q_learning_class.random_game_attempt()
    
    Output:
        Total completions: 922
        Percent complete: 92.2%
        Average moves for completion: 4.5

        Initial state:

        ['F', ' ', ' ', ' ']
        [' ', ' ', ' ', ' ']
        [' ', 'P', ' ', ' ']
        [' ', ' ', ' ', ' ']



        Move 1 up:

        ['F', ' ', ' ', ' ']
        [' ', 'P', ' ', ' ']
        [' ', 'S', ' ', ' ']
        [' ', ' ', ' ', ' ']



        Move 2 left:

        ['F', ' ', ' ', ' ']
        ['P', ' ', ' ', ' ']
        [' ', 'S', ' ', ' ']
        [' ', ' ', ' ', ' ']



        Move 3 up:

        ['P', ' ', ' ', ' ']
        [' ', ' ', ' ', ' ']
        [' ', 'S', ' ', ' ']
        [' ', ' ', ' ', ' ']



        Game Finished in 3 moves
        Least possible moves 3
    """
    def __init__(
        self,
        game_shape=SHAPE,
        learning_rate=LEARNING_RATE,
        discount_rate=DISCOUNT_RATE
    ):
        """
        Assigns object values to input args and initialises q table based on shape of game
        """
        self.game_shape = game_shape
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.q_table = np.random.uniform(size=(SHAPE + SHAPE + (4,)))
        
    def get_reward(self, new_pos, old_pos, finish_pos):
        """
        The reward function for the game.
        
        New and old pos are lists as they get edited in the game class.
        
        Args:
            new_pos (list): The next position after selecting a movement from the Q table
            old_pos (list): The "current" position of the player when selecting where to move
            finish_pos (tuple): The randomly generated end position in the game instance
        Returns:
            10: If the new position is the final position it returns a value of 10 
                as that ends the game and is the most valuable in that position
            -10: If the position is the same but not the end position
                 This is in the case the movement is trying to go outside of the game bounds
                 e.g. Moving up or left in position (0,0)
            dist_diff: The change in distance between the new/old position and the final position
                       Returns a positive value if the location is closer to the end        
        """
        
        if tuple(new_pos) == finish_pos:
            return 10
    
        new_dist = np.sqrt((new_pos[0] - finish_pos[0])**2 + (new_pos[1] - finish_pos[1])**2)

        old_dist = np.sqrt((old_pos[0] - finish_pos[0])**2 + (old_pos[1] - finish_pos[1])**2)

        dist_diff = old_dist - new_dist

        if dist_diff == 0:
            return -10

        else:
            return dist_diff
        
    def step(self, game, action, draw=False):
        """
        A singular step for updating the Q table
        
        Args:
            game (object): The game instance to perform a step on
            action (int): The action to perform within the game, in this case refer to game.actions
        """
        if draw:
            game.draw()

        old_pos = copy.deepcopy(game.player_pos)
        game.actions[action]()
        new_pos = game.player_pos
        reward_value = self.get_reward(new_pos=new_pos, old_pos=old_pos, finish_pos=game.end_pos)

        q_old = self.q_table[tuple(old_pos) + game.end_pos + (action,)]

        max_transition_q = np.max(self.q_table[tuple(new_pos) + game.end_pos])

        q_new = q_old + self.learning_rate * (reward_value + self.discount_rate * max_transition_q - q_old)

        self.q_table[tuple(old_pos) + game.end_pos + (action,)] = q_new


        if draw:
            game.draw()
    
    def episode(self, max_steps, draw=False):
        """
        Similar to an Epoch in normal machine learning/deep learning
        
        Args:
            max_steps (int): The maximum allowed moves before moving onto another game
            draw (bool): Whether to draw during the episode
        """
        game = TestGame(shape=self.game_shape)
        for i in range(max_steps):
            if not game.done:
                best_action = np.argmax(self.q_table[tuple(game.player_pos) + game.end_pos])

                self.step(game, best_action)
            else:
                return True, i+1

        return False, 10

    def fit_q_table(self, epochs, steps_per_episode, verbose=0):
        """
        The function to train the Q table
        
        Args:
            epochs (int): The amount of episodes to run in the fitting process
            steps_per_episode (int): The amount of allowed moves in each episode
            verbose (bool): Whether to print when an episode is completed
        """
        completed_runs = 0
        num_moves = []
        for i in range(epochs):
            draw = not bool(i % 10)
            ep = self.episode(
                max_steps=steps_per_episode,
                draw=draw
            )
            if ep[0]:
                completed_runs +=1
                num_moves.append(ep[1])
                if verbose:
                    print(f'Done on episode {i+1} in {ep[1]} moves')
        print(f'Total completions: {completed_runs}')
        print(f'Percent complete: {100*(completed_runs/epochs)}%')
        print(f'Average moves for completion: {np.mean(num_moves)}')

    def random_game_attempt(self, max_moves=10):
        """
        Uses the Q table of the class to attempt a randomly initialised game 
        
        Args:
            max_moves (int): The maximum allowed moves for the attempt of the game
                             If the Q table is trained for a smaller amount of time
                             This is prone to letting it run around in circles or constantly into walls
        """
        game = TestGame()
        
        least_possible = abs(game.start_pos[0] - game.end_pos[0]) + abs(game.start_pos[1] - game.end_pos[1])
        
        print('Initial state:\n')
        game.draw()
        print('\n\n')
        for i in range(max_moves):

            choice = np.argmax(self.q_table[tuple(game.player_pos) + game.end_pos])

            game.actions[choice]()
            print(f'Move {i+1} {game.actions[choice].__name__}:\n')
            game.draw()
            print('\n\n')

            if game.done:
                print(f'Game Finished in {i+1} moves')
                print(f'Least possible moves {least_possible}')
                return
        
        print(f'Ended after {max_moves} moves.')
        print(f'Least possible moves {least_possible}')

In [4]:
q_learning_class = GameLearner()

In [5]:
q_learning_class.fit_q_table(epochs=1000, steps_per_episode=MAX_EPISODE_LENGTH)

Total completions: 947
Percent complete: 94.69999999999999%
Average moves for completion: 4.536430834213305


In [6]:
q_learning_class.random_game_attempt()

Initial state:

[' ', 'P', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', 'F', ' ']
[' ', ' ', ' ', ' ']



Move 1 down:

[' ', 'S', ' ', ' ']
[' ', 'P', ' ', ' ']
[' ', ' ', 'F', ' ']
[' ', ' ', ' ', ' ']



Move 2 right:

[' ', 'S', ' ', ' ']
[' ', ' ', 'P', ' ']
[' ', ' ', 'F', ' ']
[' ', ' ', ' ', ' ']



Move 3 right:

[' ', 'S', ' ', ' ']
[' ', ' ', ' ', 'P']
[' ', ' ', 'F', ' ']
[' ', ' ', ' ', ' ']



Move 4 down:

[' ', 'S', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', 'F', 'P']
[' ', ' ', ' ', ' ']



Move 5 left:

[' ', 'S', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', 'P', ' ']
[' ', ' ', ' ', ' ']



Game Finished in 5 moves
Least possible moves 3


In [7]:
small_epoch_q_learner = GameLearner()
small_epoch_q_learner.fit_q_table(epochs=20, steps_per_episode=10)

Total completions: 4
Percent complete: 20.0%
Average moves for completion: 6.75


In [8]:
small_epoch_q_learner.random_game_attempt()

Initial state:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 1 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 2 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 3 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 4 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 5 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 6 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 7 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 8 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'P']



Move 9 down:

[' ', ' ', ' ', ' ']
[' ', 'F', ' ', ' ']
[' ', ' ', ' ', ' ']
[' ', ' ', '